# Debug prediction. Why does is take so long man?!?!?!

In [1]:
from Pointcloud.Modules.Manager import Manager
from Pointcloud.Modules.Model import custom_cos_loss
from Pointcloud.Modules.Noise import Noise
from Pointcloud.Modules.Object import Pointcloud
from Pointcloud.Modules.Preprocessor import Preprocessor
from Pointcloud.Modules.Visualize import *

from meshplot import plot as mp_plot
import random
import torch_geometric as tg
import torch

# EXAMPLE_OBJ_FILE = "common-3d-test-models-master/stanford-bunny.obj"
# EXAMPLE_OBJ_FILE = "common-3d-test-models-master/fandisk.obj"
# EXAMPLE_OBJ_FILE = "common-3d-test-models-master/beetle.obj"
EXAMPLE_OBJ_FILE = "PatchDataset/raw/untitled.obj"

c:\Users\ruben\miniconda3\envs\PCDenoiser\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: '[WinError 127] The specified procedure could not be found'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
c:\Users\ruben\miniconda3\envs\PCDenoiser\lib\site-packages\torchaudio\backend\utils.py:62: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [2]:
pointcloud = Pointcloud.loadObj(EXAMPLE_OBJ_FILE)
pp = Preprocessor(pointcloud)
noise = Noise(pp.graph)
noise.generateNoise(0.01, 0, 0)
pp.graphBuilder.generateNormals()
manager = Manager()
prediction, RInv = manager.predict(pp, "tb_logs/testmodel/version_181/checkpoints/0-epoch19-00.45.ckpt")

Creating MST..: 100%|██████████| 14348/14348 [00:00<00:00, 48494.03it/s]
Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Model size: 4.440509796142578 MB
Train Dataset size: 22.982025146484375 MB


Creating patches in list: 100%|██████████| 2048/2048 [00:01<00:00, 1994.26it/s]
Restoring states from the checkpoint path at tb_logs/testmodel/version_181/checkpoints/0-epoch19-00.45.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at tb_logs/testmodel/version_181/checkpoints/0-epoch19-00.45.ckpt


Predicting: |          | 0/? [00:00<?, ?it/s]

PREDICT Profiler Report
Profile stats for: records
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         6.57%      48.001ms        99.82%     728.973ms      36.449ms      41.254ms         5.65%     729.216ms      36.461ms            20  
[pl][profile][Strategy]SingleDeviceStrategy.predict_...         7.87%      57.460ms        75.40%     550.627ms      27.531ms      42.432ms         5.81%   

In [15]:
_prediction = prediction
gt = pp.graph.gt_n
_scale = 20
sim = custom_cos_loss(pp.graph.n, gt)
new_sim = custom_cos_loss(_prediction, gt)
start = pp.graph.pos.numpy()
endpred = start + _prediction.numpy() / _scale
endgt = start + gt.numpy() / _scale
endinput = start + pp.graph.n.numpy() / _scale
color = torch.max(torch.nn.functional.cosine_similarity(_prediction, gt), torch.nn.functional.cosine_similarity(_prediction, -gt))
print(f"Current average custom cosine loss: {sim}\nPredicted average custom cosine loss: {new_sim}\nHow much worse it got (factor): {new_sim / sim}\nMin error: {color.min()}\nMax error: {color.max()}")
print(f"Biggest errors: {1 - color.sort(dim=0).values[:5]}")
plot = mp_plot(start, c=color.numpy(), shading={"point_size": 0.1})
# plot.add_lines(start, endpred, shading={"line_color": "green"})
# plot.add_lines(start, endgt, shading={"line_color": "yellow"})
# plot.add_lines(start, endinput, shading={"line_color": "purple"})

Current average custom cosine loss: 0.003881698241457343
Predicted average custom cosine loss: 0.0032612006179988384
How much worse it got (factor): 0.8401479125022888
Min error: 0.3051972985267639
Max error: 0.9999991059303284
Biggest errors: tensor([0.6948, 0.5951, 0.0592, 0.0498, 0.0334])


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0035199…

In [ ]:
pp.pointcloud.v.size(0)//2

1024

In [ ]:
graph_list = tg.data.Batch.to_data_list(torch.load("PatchDataset/processed/untitled_gaussian_0.02_nonfeature.pt", "cpu"))

In [21]:
R = RInv.transpose(1, 2)
graph_list = pp.graph_list
i = random.randint(0, len(graph_list) - 1)
patch = graph_list[i]
_x = patch.x
_pos = _x[:, :3]
center_dist = (_pos - _pos.mean(dim=0)).norm(dim=1)
_pos0 = _pos[center_dist.min(dim=0).indices]
mean_dist = center_dist.mean()
print(patch)
print(_pos.norm(dim=1).mean(dim=0))
plot = visSample(patch)
plot.add_lines(_pos0.numpy(), (_pos0 + R[i] @ prediction[i] * mean_dist).squeeze().numpy(), shading={"line_color": "orange"})
# visGraph(pp.graph)

Data(x=[18, 8], edge_index=[2, 96], y=[1, 3])
tensor(0.5949)


Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.014391…

4

In [3]:
from pathlib import Path
import torch
loaded = torch.load(Path("PatchDataset/processed/Super_Cool_Split_Bro.split"))

In [7]:
N = sum([len(x) for x in loaded])
data = torch.zeros(N, dtype=torch.long)
for i, v in enumerate(loaded):
    data[v] = i
print(data.unique(return_counts=True))

(tensor([0, 1, 2, 3, 4, 5]), tensor([3337,  254,  261, 1540,  513,  515]))


In [15]:
graphs, RInv = pp.getPatches()
dl = tg.loader.DataLoader(
    dataset=graphs,
    batch_size=32,
    shuffle=False,
    num_workers=4,
    persistent_workers=True
)
from_checkpoint = "tb_logs/testmodel/version_181/checkpoints/0-epoch19-00.45.ckpt"
batchedOutput = manager.trainer.predict(manager.model, dl, ckpt_path=from_checkpoint)

Process edges for ring 2: 100%|██████████| 1/1 [00:00<00:00, 500.27it/s]


Creating patches in list: 100%|██████████| 2048/2048 [00:00<00:00, 2197.49it/s]
Restoring states from the checkpoint path at tb_logs/testmodel/version_181/checkpoints/0-epoch19-00.45.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at tb_logs/testmodel/version_181/checkpoints/0-epoch19-00.45.ckpt


Predicting: |          | 0/? [00:00<?, ?it/s]

PREDICT Profiler Report
Profile stats for: records
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg    # of Calls  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  
                                          ProfilerStep*         6.96%      46.924ms        99.89%     673.666ms      33.683ms      40.864ms         6.06%     673.941ms      33.697ms            20  
[pl][profile][Strategy]SingleDeviceStrategy.predict_...         7.23%      48.760ms        72.24%     487.203ms      24.360ms      33.973ms         5.04%   

In [42]:
randint = torch.randperm(len(graphs))[0]
plot = visSample(graphs[randint])
plot.add_lines(torch.zeros((3,)).numpy(), torch.cat(batchedOutput, dim=0)[randint].numpy())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(-0.001108…

4